<a href="https://colab.research.google.com/github/matpg/RL-Agent-for-Unreal-Engine-4/blob/main/script_learning_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Entrenamiento del Agente

### Descarga de librerias secundarias

In [1]:
# Stable Baselines only supports tensorflow 1.x for now
%tensorflow_version 1.x
!apt-get update
!apt-get install swig cmake libopenmpi-dev zlib1g-dev ffmpeg freeglut3-dev xvfb
!pip install stable-baselines[mpi] --upgrade
!pip install pybullet
!pip install box2d box2d-kengz pyyaml pytablewriter optuna scikit-optimize


TensorFlow 1.x selected.
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [40.7 kB]
Get:12 http:

### Descarga de librerias principales

In [2]:
!git clone https://github.com/araffin/rl-baselines-zoo
%cd rl-baselines-zoo/
!pip install gym-minigrid

Cloning into 'rl-baselines-zoo'...
remote: Enumerating objects: 44, done.
remote: Counting objects: 100% (44/44), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 1840 (delta 20), reused 26 (delta 12), pack-reused 1796
Receiving objects: 100% (1840/1840), 375.67 MiB | 40.60 MiB/s, done.
Resolving deltas: 100% (1085/1085), done.
Checking out files: 100% (333/333), done.


### Montar Folder de Drive para subida de archivos

In [5]:
# MONTANDO FOLDER COMPARTIDO DE DRIVE
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


### Implementación de ajustes de entorno, algoritmo y cambios de script

In [6]:
# moving the mod crossing file (U MAZE MODELED) to the destination path
%rm "/usr/local/lib/python3.6/dist-packages/gym_minigrid/envs/crossing.py"
%cp "/content/drive/MyDrive/Colab Notebooks/modelo ts agentnet/ajustes de entorno y modelo/crossing.py" "/usr/local/lib/python3.6/dist-packages/gym_minigrid/envs"

#SIMULTANEAMENTE SE DEBE REEMPLAZAR LA INFORMACIÓN DEL ENTORNO EN LOS HIPERPARAMETROS PPO2 DEL PAQUETE ZOO
%rm "/content/rl-baselines-zoo/hyperparams/ppo2.yml"
%cp "/content/drive/MyDrive/Colab Notebooks/modelo ts agentnet/ajustes de entorno y modelo/ppo2.yml" "/content/rl-baselines-zoo/hyperparams"

#tambien, actualizar el archivo train.py con cambios en los env wrappers.
%rm "/content/rl-baselines-zoo/train.py"
%cp "/content/drive/MyDrive/Colab Notebooks/modelo ts agentnet/ajustes de entorno y modelo/train.py" "/content/rl-baselines-zoo"

### Ejecución de Entrenamiento con algoritmo PPO2 e hiper parámetros definidos

In [ ]:
#ENTRENAMIENTO PARA LA RED CON ALGORITMO PPO2 PARA EL ENTORNO DE PUZZLES CLASICOS
!python train.py --algo ppo2 --env MiniGrid-SimpleCrossingS9N1-v0 --gym-packages gym_minigrid


In [ ]:
#ENTRENAMIENTO PARA LA RED CON ALGORITMO PPO2 PARA EL ENTORNO DE PUZZLES DE UE4
!python train.py --algo ppo2 --env MiniGrid-SimpleCrossingEnvUmaze-v0 --gym-packages gym_minigrid


In [ ]:
#ENTRENAMIENTO PARA LA RED CON ALGORITMO PPO2 PARA EL ENTORNO GRANDES DE PUZZLES CLASICOS
!python train.py --algo ppo2 --env MiniGrid-SimpleCrossingS11N5-v0 --gym-packages gym_minigrid


In [ ]:
#ENTRENAMIENTO PARA LA RED CON ALGORITMO PPO2 PARA EL ENTORNO GRANDE DE PUZZLES DE UE4
!python train.py --algo ppo2 --env MiniGrid-SimpleCrossingS9N3-v0 --gym-packages gym_minigrid


### Otras funciones

In [ ]:
# Set up display; otherwise rendering will fail
import os
os.system("Xvfb :1 -screen 0 1024x768x24 &")
os.environ['DISPLAY'] = ':1'

In [ ]:
!pip install pyglet==1.3.1  # pyglet v1.4.1 throws an error

In [ ]:
!python -m utils.record_video --algo ppo2 --env MiniGrid-SimpleCrossingEnvUmaze-v0 --gym-packages gym_minigrid --exp-id 0 -f logs/ -n 1000

In [ ]:
import base64
from pathlib import Path

from IPython import display as ipythondisplay

def show_videos(video_path='', prefix=''):
  """
  Taken from https://github.com/eleurent/highway-env

  :param video_path: (str) Path to the folder containing videos
  :param prefix: (str) Filter the video, showing only the only starting with this prefix
  """
  html = []
  for mp4 in Path(video_path).glob("{}*.mp4".format(prefix)):
      video_b64 = base64.b64encode(mp4.read_bytes())
      html.append('''<video alt="{}" autoplay 
                    loop controls style="height: 400px;">
                    <source src="data:video/mp4;base64,{}" type="video/mp4" />
                </video>'''.format(mp4, video_b64.decode('ascii')))
  ipythondisplay.display(ipythondisplay.HTML(data="<br>".join(html)))

In [ ]:
show_videos(prefix='a2c')